In [1]:
import numpy as np
import pandas as pd

filename='C:\\题面\\市区域.csv'
f=open(filename,'r',errors='ignore',encoding='UTF8')
df=pd.read_csv(f,header=None)
f.close()

df.columns=['cityid','cityname','provinceid']

filename='C:\\题面\\省区域.csv'
f=open(filename,'r',errors='ignore',encoding='UTF8')
df0=pd.read_csv(f,header=None)
f.close()

df0.columns=['provinceid','provincename','parentid']

df_c_p=pd.merge(df,df0,how='inner',on='provinceid') 
print(df_c_p)

filename='C:\\题面\\县区区域.csv'
f=open(filename,'r',errors='ignore',encoding='UTF8')
df2=pd.read_csv(f,header=None)
f.close()
df2.columns=['countryid','countryname','cityid']

filename='C:\\题面\\基站.csv'
f=open(filename,'r',errors='ignore',encoding='UTF8')
df3=pd.read_csv(f,header=None)
f.close()

df3.columns=['EnodebId','EnodebName','EnodebType','countryid','EnodebStatus']
print(df3.head())

# ------第一道题-----
print(df.count()) #市的数量
print(df2.count()) #县区的数量
print(df3[df3['EnodebStatus'].isin(['在网'])].count()) #在网状态的基站数
print(df3.count()) #整体的基站数

#------第二道题-------------
df_c_p_p=pd.merge(df_c_p,df2,how='inner',on='cityid')  
df_c_p_p_e=pd.merge(df_c_p_p,df3,how='inner',on='countryid')

filename='C:\\题面\\告警.csv'
f=open(filename,'r',errors='ignore',encoding='UTF8')
df4=pd.read_csv(f,header=None)
f.close()

df4.columns=['id1','alertid','date1','date2','EnodebName']
df_all=pd.merge(df_c_p_p_e,df4,how='inner',on='EnodebName') #连接成为一个大表 df_all

A1=df_all.groupby('cityname').cityid.count() #分组统计 cityname 的告警次数
dict1=dict(A1) #将统计结果转换为字典
A2=sorted(dict1.items(),key=lambda e:e[1],reverse=True) #对字典里的value排序
print(A2[0:3]) #显示前三个城市与告警次数





    cityid cityname  provinceid provincename  parentid
0     1012      周口市        1000          河南省       999
1     1014     平顶山市        1000          河南省       999
2     1002      商丘市        1000          河南省       999
3     1016      新乡市        1000          河南省       999
4     1009      南阳市        1000          河南省       999
5     1011      信阳市        1000          河南省       999
6     1003      开封市        1000          河南省       999
7     1018      鹤壁市        1000          河南省       999
8     1005      安阳市        1000          河南省       999
9     1004      洛阳市        1000          河南省       999
10    1001      郑州市        1000          河南省       999
11    1015     三门峡市        1000          河南省       999
12    1017      许昌市        1000          河南省       999
13    1010      漯河市        1000          河南省       999
14    1007      焦作市        1000          河南省       999
15    1006      濮阳市        1000          河南省       999
16    1013     驻马店市        1000          河南省       999
17    1008

In [2]:
#第二题
from datetime import datetime
dt = datetime(2018, 8, 8, 0, 0,0)
dt2 = datetime(2018, 8, 9, 0, 0,0)
dict1={}
s=0
for i in range(df_all.cityid.count()):
    a1=df_all.iloc[i]
    cday = datetime.strptime(a1.date1, '%Y/%m/%d %H:%M:%S')
    if cday>=dt and cday<dt2:
        s=s+1
        if a1.cityname in dict1:
            dict1[a1.cityname]=dict1[a1.cityname]+1
        else:
            dict1[a1.cityname]=1

A2=sorted(dict1.items(),key=lambda e:e[1],reverse=True) #对字典里的value排序
print(A2[0:3]) #显示前三个城市与告警次数

[('洛阳市', 47054), ('南阳市', 5559), ('信阳市', 4935)]


In [3]:
# 为df_all 增加三列(开始相对时间,结束相对时间, 处理时间) 单位:秒
tlist1=[]
tlist2=[]

for i in range(df_all.cityid.count()):
    a1=df_all.iloc[i]
    cday = datetime.strptime(a1.date1, '%Y/%m/%d %H:%M:%S')
    tlist1.append(cday.timestamp()-dt.timestamp())
    cday2 = datetime.strptime(a1.date2, '%Y/%m/%d %H:%M:%S')
    tlist2.append(cday2.timestamp()-dt.timestamp())
print(len(tlist1),len(tlist2))

171180 171180


In [4]:
df_all["nTime1"]=tlist1
df_all["nTime2"]=tlist2
df_all["nTimeDiff"]=df_all["nTime2"]-df_all["nTime1"]

In [5]:
# 定义函数, 根据相对秒数, 还原日期
def computedate(ts):
    x = datetime(2018, 8, 8, 0, 0,0)
    D=x.timestamp()+ts
    A=datetime.fromtimestamp(D)
    return A.strftime("%Y/%m/%d %H:%M:%S")

In [6]:
# 第三道题 3.1
dict3={}
dict3A={}
for i in range(0,86400,60):
    dict2=dict(df_all[(df_all.alertid=='007-103-00-040012') &  (df_all.nTime1<i) & (df_all.nTime2>i)].groupby('cityid')['cityname'].count())
    # 当前每个城市 告警次数
    for k in dict2:
        if dict2[k]>=60:
            if k not in dict3:
                dict3[k]={}
                dict3[k]['alarm']='二级'
                dict3[k]['starttime']=i
                #print(k,dict2[k],'begin')
        elif dict2[k]<60:
            if k in dict3: 
                dict3[k]['endtime']=i
                print(k,dict3[k],dict2[k])
                dict3.pop(k)
        if dict2[k]>=150:
            if k not in dict3A:
                dict3A[k]={}
                dict3A[k]['alarm']='一级'
                dict3A[k]['starttime']=i
                #print(k,dict2[k],'begin')
        elif dict2[k]<150:
            if k in dict3A: 
                dict3A[k]['endtime']=i
                print(k,dict3A[k],dict2[k])
                dict3A.pop(k)

    

1005 {'alarm': '一级', 'starttime': 7440, 'endtime': 7620} 112
1005 {'alarm': '二级', 'starttime': 7440, 'endtime': 7680} 48


In [7]:
# 第三道题 3.2 精准锁定时间到秒
dict3={}
dict3A={}
targetcity='1005'
time1=7440
time2=7680
for i in range(time1-60,time2+60,1):
    dict2=dict(df_all[(df_all.cityid.isin([targetcity])) &  (df_all.alertid=='007-103-00-040012') &  (df_all.nTime1<i) & (df_all.nTime2>i)].groupby('cityid')['cityname'].count())
    # 当前每个城市 告警次数
    for k in dict2:
        if dict2[k]>=60:
            if k not in dict3:
                dict3[k]={}
                dict3[k]['alarm']='二级'
                dict3[k]['starttime']=computedate(i)
                #print(k,dict2[k],'begin')
        elif dict2[k]<60:
            if k in dict3: 
                dict3[k]['endtime']=computedate(i)
                print(k,dict3[k],dict2[k])
                dict3.pop(k)
        if dict2[k]>=150:
            if k not in dict3A:
                dict3A[k]={}
                dict3A[k]['alarm']='一级'
                dict3A[k]['starttime']=computedate(i)
                #print(k,dict2[k],'begin')
        elif dict2[k]<150:
            if k in dict3A: 
                dict3A[k]['endtime']=computedate(i)
                print(k,dict3A[k],dict2[k])
                dict3A.pop(k)

    

1005 {'alarm': '一级', 'starttime': '2018/08/08 02:03:54', 'endtime': '2018/08/08 02:06:03'} 149
1005 {'alarm': '二级', 'starttime': '2018/08/08 02:03:27', 'endtime': '2018/08/08 02:07:59'} 49


In [8]:
#第四道题

dict3={}
dict3A={}
for i in range(0,86400,60):
    dict2=dict(df_all[(df_all.alertid=='007-103-00-040012') &  (df_all.nTime1<i) & (df_all.nTime2>i)].groupby('cityid')['nTimeDiff'].sum())
    # 当前每个城市 告警次数
    for k in dict2:
        if dict2[k]>=432000:
            if k not in dict3:
                dict3[k]={}
                dict3[k]['alarm']='二级'
                dict3[k]['starttime']=computedate(i)
                dict3[k]['startvalue']=dict2[k]
        elif dict2[k]<432000:
            if k in dict3: 
                dict3[k]['endtime']=computedate(i)
                print(k,dict3[k],dict2[k])
                dict3.pop(k)
        if dict2[k]>=864000:
            if k not in dict3A:
                dict3A[k]={}
                dict3A[k]['alarm']='一级'
                dict3A[k]['starttime']=computedate(i)
                dict3A[k]['startvalue']=dict2[k]
                #print(k,dict2[k],'begin')
        elif dict2[k]<864000:
            if k in dict3A: 
                dict3A[k]['endtime']=computedate(i)
                print(k,dict3A[k],dict2[k])
                dict3A.pop(k)


1004 {'alarm': '一级', 'starttime': '2018/08/08 02:51:00', 'startvalue': 864511.0, 'endtime': '2018/08/08 06:47:00'} 775531.0
1004 {'alarm': '二级', 'starttime': '2018/08/08 00:00:00', 'startvalue': 808781.0, 'endtime': '2018/08/08 07:08:00'} 2741.0


In [18]:
# 第五题
# 以城市为准的基站数量
dict2=dict(df_c_p_p_e[(df_c_p_p_e.EnodebStatus.isin(['在网']))].groupby('cityid')['cityname'].count())
# 以城市为准统计基站报错时常(累计)
dict3=dict(df_all[(df_all.alertid=='007-103-00-040012') &  (df_all.nTime1>=0) & (df_all.nTime1<86400)].groupby('cityid')['nTimeDiff'].sum())
dict4={}
for k in dict3:
    dict4[k]=1-dict3[k]/(dict2[k]*86400)
A4=sorted(dict4.items(),key=lambda e:e[1],reverse=False) #对字典里的value排序
print(A4[0]) #显示最差的那个城市的基站可用率

(1018, 0.9991195023148148)
